# Chapter 5 오차역전파법

수치 미분은 시간이 오래 걸리는 단점이 있다. 오차역전파법(backpropagation)은 효율적 계산이 가능하다.

- 수식을 통한 이해
- 계산 그래프를 통한 이해 ★

참고
- http://karpathy.github.io/
- Stanford CS231n

## 5.1 계산 그래프

**계산 그래프(computational graph)**는 계산 과정을 그래프로 나타낸 것이다. 그래프는 **노드(node)**와 **에지(edge)**로 표현된다.

### 5.1.1 계산 그래프로 풀다

<img src="https://t1.daumcdn.net/cfile/tistory/997ED34B5B98F5F235">
<center><small>▲ 간단한 계산 그래프</small></center>

계산 그래프 문제 흐름
1. 계산 그래프를 구성한다.
2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다.

계산을 왼쪽에서 오른쪽으로 진행하는 단계를 **순전파(forward propagation)**이라고 하고 반대 방향을 **역전파(backward propagation)**이라고 한다.

### 5.1.2 국소적 계산

계산 그래프는 국소적 계산을 전파해서 최종 결과를 얻을 수 있다는 특징이 있다. 즉, 다른 부분은 상관하지 않고 자신과 관계된 정보만 출력할 수 있다. 이러한 특징에 따라 각 노드는 자신과 관계된 계산에만 집중하면 된다.

<img src="https://t1.daumcdn.net/cfile/tistory/991C9E495B98F60F1D">
<center><small>▲ 국소적 계산의 예</small></center>

### 5.1.3 왜 계산 그래프로 푸는가?

계산 그래프의 이점
- 국소적 계산으로 복잡한 문제를 단순화할 수 있다.
- 역전파를 통해 (다수의) 미분을 효율적으로 계산할 수 있다.

<img src="https://t1.daumcdn.net/cfile/tistory/997E914D5B98F62826">
<center><small>▲ 역전파를 통한 미분</small></center>

## 5.2 연쇄법칙(chain rule)

### 5.2.1 계산 그래프의 역전파

<img src="https://t1.daumcdn.net/cfile/tistory/999FD3425B98F63F1A">

국소적 미분은 상류에서 전달된 값과 곱해져서 앞쪽 노드로 전달된다.

### 5.2.2 연쇄법칙이란?

연쇄법칙은 합성 함수의 미분이 각 구성 함수의 미분의 곱으로 나타낸다는 성질을 이용한다.

예를 들어, $z = (x + y)^2$가 있을때 $x$에 대한 $z$의 미분은 다음과 같이 나타낼 수 있다.

$$\frac {\partial z} {\partial x} = \frac {\partial z} {\partial t} \frac {\partial t} {\partial x}$$

$$\frac {\partial z} {\partial t} = 2t$$

$$\frac {\partial t} {\partial x} = 1$$

$$\frac {\partial z} {\partial x} = \frac {\partial z} {\partial t} \frac {\partial t} {\partial x} = 2t \cdot 1 = 2(x + y)$$

### 5.2.3 연쇄법칙과 계산 그래프

<img src="https://t1.daumcdn.net/cfile/tistory/997387465B98F65D13">

## 5.3 역전파

### 5.3.1 덧셈 노드의 역전파

덧셈 노드의 역전파는 입력된 값을 그대로 다음 노드로 보낸다.

<img src="https://t1.daumcdn.net/cfile/tistory/99FB57455B98F67407">

### 5.3.2 곱셈 노드의 역전파

곱셈 노드의 역전파는 순전파 때의 입력 신호들을 서로 바꾼 값을 곱해서 하류로 보낸다. 그래서 곱셈 노드를 구현할 때는 순전파의 입력 신호를 변수에 저장한다.

<img src="https://t1.daumcdn.net/cfile/tistory/99E3EF435B98F69309">

### 5.3.3 사과 쇼핑의 예

<img src="https://t1.daumcdn.net/cfile/tistory/99AACA445B98F6A61E">
<center><small>▲ 사과 쇼핑의 역전파 예</small></center>

<img src="https://t1.daumcdn.net/cfile/tistory/99499E4E5B98F6C10E">
<center><small>▲ 사과와 귤 쇼핑의 역전파 예</small></center>

## 5.4 단순한 계층 구현하기

### 5.4.1 곱셈 계층

In [2]:
class MulLayer:
    
    def __init__(self):
        self.x = None
        self.y = None
        
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

- 사과 쇼핑 구현

In [13]:
apple = 100
apple_num = 2
tax = 1.1

# 계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price) # 다들 오차 나는지?

220.00000000000003


In [14]:
# 역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


### 5.4.2 덧셈 계층

In [15]:
class AddLayer:
    
    def __init__(self):
        pass
    
    
    def forward(self, x, y):
        out = x + y
        return out
    
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        
        return dx, dy

덧셈 계층은 그저 상류에서 내려온 미분을 하류로 흘러보내기만 하면 되기 때문에 따로 초기화할 필요가 없다.

- 사과 2개와 귤 3개를 사는 상황

In [18]:
apple_num = 2
apple = 100
mandarin_num = 3
mandarin = 150
tax = 1.1

mul_apple_layer = MulLayer()
mul_mandarin_layer = MulLayer()
add_fruit_layer = AddLayer()
mul_tax_layer = MulLayer()

apple_price = mul_apple_layer.forward(apple, apple_num)
mandarin_price = mul_mandarin_layer.forward(mandarin, mandarin_num)
fruit_price = add_fruit_layer.forward(apple_price, mandarin_price)
total_price = mul_tax_layer.forward(fruit_price, tax)

print(total_price)

dtotal_price = 1
dfruit_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dmandarin_price = add_fruit_layer.backward(dfruit_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dmandarin, dmandarin_num = mul_mandarin_layer.backward(dmandarin_price)

print(dapple, dapple_num, dmandarin, dmandarin_num)

715.0000000000001
2.2 110.00000000000001 3.3000000000000003 165.0


## 5.5 활성화 함수 계층 구현하기

### 5.5.1 ReLU 계층

- ReLU 수식

$$y = 
\begin{cases}
x \ (x > 0) \\
0 \ (x \leq 0)
\end{cases}$$

- ReLU 미분

$$\frac {\partial y}{\partial x} = 
\begin{cases}
1 \ (x > 0) \\
0 \ (x \leq 0)
\end{cases}$$

<img src="https://t1.daumcdn.net/cfile/tistory/99E517485B98F6E504">
<center><small>▲ ReLU 계산 그래프</small></center>

In [21]:
import numpy as np

In [19]:
class Relu:
    
    def __init__(self):
        self,mask = None    # 입력 원소가 0 이하인 인덱스는 True, 0보다 큰 경우 False 유지
        
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return 0
    
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [22]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [24]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


mask 인스턴스 변수를 써서 mask의 원소가 True인 곳은 상류에서 전파된 미분값을 0으로 바꾼다.

### 5.5.2 Sigmoid 계층

- 시그모이드 수식

$$y = \frac 1 {1+exp(-x)}$$

- '/' 노드, $y = \frac 1 x$ 미분

$$\begin{align}
\frac {\partial y} {\partial x} & = -\frac 1 {x^2} \\
& = -y^2 \\
\end{align}$$

- exp 노드 미분

$$\frac {\partial y} {\partial x} = exp(x)$$


<img src="https://t1.daumcdn.net/cfile/tistory/999E3B4B5B98F72021">
<center><small>▲ 시그모이드 순전파/역전파</small></center>

- sigmoid 미분

$$\begin{align}
\frac {\partial y} {\partial x} & = y^2exp(-x) \\
& = \frac 1 {(1 + exp(-x))^2} exp(-x) \\
& = \frac 1 {1 + exp(-x)} \frac {exp(-x)} {1+exp(-x)} \\
& = y(1-y)
\end{align}$$

시그모이드 계층의 역전파는 순전파의 출력만으로 계산할 수 있다.

In [25]:
class Sigmoid:
    
    def __init__(self):
        self.out = None
    
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        
        return out
    
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        
        return dx

구현에서 순전파의 출력을 out 인스턴스 변수에 저장해 놓고 역전파 계산할 때 사용한다.

## 5.6 Affine/Softmax 계층 구현하기

### 5.6.1 Affine 계층

행렬의 곱을 기하학에서는 **어파인 변환(affine transformation)**이라고 한다.

$$\frac {\partial L} {\partial X} = \frac {\partial L} {\partial Y} \cdot W^T$$

$$\frac {\partial L} {\partial W} = X^T \cdot \frac {\partial L} {\partial Y}$$

<img src="https://t1.daumcdn.net/cfile/tistory/994002375B98F73E05">

계산 그래프에서 각 원소의 형상에 주의해야 한다.

### 5.6.2 배치용 Affine 계층

<img src="https://t1.daumcdn.net/cfile/tistory/994510365B98F75122">

편향의 경우에는 순전파에서 각각의 데이터에 더해진다. 그래서 역전파 때는 편향의 원소에 역전파 값이 편향에 모여야 한다.

In [26]:
class Affine:
    
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

### 5.6.3 Softmax-with-Loss 계층

Softmax 계층은 출력의 합이 1이 되도록 정규화하여 출력한다.

<img src="https://t1.daumcdn.net/cfile/tistory/995A16395B98F76820">

<img src="https://t1.daumcdn.net/cfile/tistory/99EBF5395B98F7792B">
<center><small>▲ Softmax-with-Loss 계층의 계산 그래프</small></center>

<img src="https://camo.qiitausercontent.com/c879a9a466f923c0978973590908d2b1c0725592/68747470733a2f2f71696974612d696d6167652d73746f72652e73332e616d617a6f6e6177732e636f6d2f302f3139373530382f66353935633337652d323562312d383666392d356438632d3937343532343461626633372e706e67" width=450>
<center><small>▲ Softmax-with-Loss 계층 계산 그래프 간소화</small></center>

Softmax 계층의 역전파 결과에서 중요한 점은 Softmax 계층의 출력과 정답 레이블의 차이로, 신경망의 현재 출력과 정답 레이블의 오차를 그래로 드러낸다는 것이다.

참고로 항등 함수의 손실 함수로 평균 제곱 오차를 사용하는데 이 때의 역전파 값도 위와 동일하다.

In [28]:
# 소프트맥스 오버플로 개선 버전
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y

In [29]:
# 데이터가 1개나 그 이상의 배치로 주어지는 경우
def cross_entropy_error(y, t):
    # y가 1차원, 즉 하나의 데이터일 경우 shape을 바꿔준다.
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [30]:
class SoftmaxWithLoss:
    
    def __init__(self):
        self.loss = None   # 손실
        self.y = None      # softmax 출력
        self.t = None      # 정답 레이블(원-핫 벡터)
        
    
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx

## 5.7 오차역전파법 구현하기

### 5.7.1 신경망 학습의 전체 그림 (생략)

### 5.7.2 오차역전파법을 적용한 신경망 구현하기

In [31]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

In [37]:
class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params["W1"], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    
    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return self.lastLayer.forward(y, t)
    
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : 
            t = np.argmax(t, axis=1)
            
        accuracy = np.sum(y == t) / float(x.shape[0])
        
        return accuracy
    
    
    # x: 입력 데이터, t: 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)
        
        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

### 5.7.3 오차역전파법으로 구한 기울기 검증하기

기울기 구하는 방법
- 수치 미분: 구현은 간단하지만 느리다
- 해석적 방법: 오차역전파법 이용해서 매개변수 많아도 빠르게 계산 가능

실제 학습을 할 땐 계산이 빠른 오차역전파법을 이용하고 수치 미분은 오차역전파법을 정확하게 구현했는지 확인하는 용도로 사용한다. 두 방식으로 기울기가 일치하는 것을 확인하는 작업을 **기울기 확인(gradient check)**라고 한다.

In [38]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

In [42]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 가 가중치의 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다.
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ": " + str(diff))

W1: 2.0655630540316686e-10
b1: 1.1163263739284333e-09
W2: 7.232095981457576e-08
b2: 1.4492399771914855e-07


### 5.7.4 오차역전파법을 사용한 학습 구현하기

In [43]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

In [45]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 오차역전파법으로 기울기를 구한다.
    grad = network.gradient(x_batch, t_batch)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.11236666666666667 0.1135
0.7885 0.7957
0.8772 0.8808
0.8986666666666666 0.9036
0.9084666666666666 0.9127
0.9139166666666667 0.9173
0.9194333333333333 0.9213
0.9230166666666667 0.9252
0.9273333333333333 0.9294
0.9311333333333334 0.9312
0.9345166666666667 0.9336
0.9368833333333333 0.9372
0.9398333333333333 0.9386
0.94185 0.9404
0.9450833333333334 0.9424
0.9458666666666666 0.944
0.9469 0.9467
